# Implementação de Métodos de Otimização

Vamos começar implementando um método simples.

## Método do Gradiente com busca de Armijo

$$ x^{k+1} = x^k + t_kd^k $$
$$ f(x^{k+1}) < f(x^k) + \alpha t^k \nabla f(x^k)^Td^k $$

In [1]:
function metodo_gradiente(f, ∇f, x₀)
    ϵ = 1e-4
    α = 0.5
    η = 0.5
    kmax = 1000
    
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -∇f(x)
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dot(d,∇f(x))
            t *= η
        end
        x = x + t*d
        k += 1
        if k > kmax
            ef = 1
            break
        end
    end
    return x, f(x), ef, k
end

metodo_gradiente (generic function with 1 method)

In [2]:
n = 100
Λ = linspace(1e-4, 1.0, n) # Linearly spaced vector from 1e-4 to 1.0
f(x) = 0.5*dot(x, Λ.*x); ∇f(x) = Λ.*x

x, fx, ef, k = metodo_gradiente(f, ∇f, ones(n))

([0.945442,0.00317775,1.00747e-5,3.00921e-8,8.45733e-11,2.23366e-13,5.53637e-16,1.28606e-18,2.79584e-21,5.67987e-24  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],4.4744523760589496e-5,0,561)

In [3]:
println("f(x) = $fx; ef = $ef; k = $k")

f(x) = 4.4744523760589496e-5; ef = 0; k = 561


### Upgrade da função

Uma maneira simples de limpar a função é colocar alguns parâmetros como
argumentos por palavra-chave. Assim já temos a opção de fazer mudanças na
chamada da função, mas também temos uma opção simples para outras pessoas usarem.
Se tivermos muitas funções com muitos argumentos isso pode ficar complicado,
e/ou feio. Podemos tentar melhorar um pouco a estética usando um tipo
composto (equivalente à struct, mas que permite construtores).
Outra opção é usar o pacote [Options.jl](https://github.com/JuliaLang/Options.jl).

In [4]:
function metodo_gradiente2(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -∇f(x)
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dot(d,∇f(x))
            t *= η
        end
        x = x + t*d
        k += 1
        if k > kmax
            ef = 1
            break
        end
    end
    return x, f(x), ef, k
end

metodo_gradiente2 (generic function with 1 method)

In [5]:
x, fx, ef, k = metodo_gradiente2(f, ∇f, ones(n), ϵ=1e-5, kmax=100000)
println("f(x) = $fx; ef = $ef; k = $k")

f(x) = 4.99969961220827e-7; ef = 0; k = 23025


## Melhorando o código um pouco

Estamos calculando $f(x)$ e $\nabla f(x)$ toda iteração. Vamos evitar.

In [6]:
function metodo_gradiente3(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    ef = 0
    x = copy(x₀)
    k = 0
    fx = f(x)
    ∇fx = ∇f(x)
    while norm(∇fx) > ϵ
        t = 1.0
        dt∇f = dot(∇fx,∇fx)
        while f(x - t*∇fx) > fx - α*t*dt∇f
            t *= η
        end
        x = x - t*∇fx
        fx = f(x)
        ∇fx = ∇f(x)
        k += 1
        if k > kmax
            ef = 1
            break
        end
    end
    return x, fx, ef, k
end

metodo_gradiente3 (generic function with 1 method)

In [7]:
x, fx, ef, k = metodo_gradiente3(f, ∇f, ones(n), ϵ=1e-5, kmax=100000)
println("f(x) = $fx; ef = $ef; k = $k")

f(x) = 4.99969961220827e-7; ef = 0; k = 23025


In [8]:
@time x, fx, ef, k = metodo_gradiente2(f, ∇f, ones(n));

In [9]:
@time x, fx, ef, k = metodo_gradiente3(f, ∇f, ones(n));

## CUTEst

O [CUTEst](http://ccpforge.cse.rl.ac.uk/gf/project/cutest/) ainda é o
principal repositório de testes para algoritmos de otimização não-linear.
Ele é escrito em Fortran, e também funciona com C e MatLab.

Agora também temos uma interface para Julia, o
[CUTEst.jl](https://github.com/JuliaOptimizers/CUTEst.jl). Essa interface
está em desenvolvimento, e no momento atual o ponto recomendado para uso
é [este](https://github.com/abelsiqueira/CUTEst.jl/tree/fix/issue4).

Para instalar o CUTEst (mesmo sem Julia), recomendamos o pacote
[homebrew-cutest](https://github.com/optimizers/homebrew-cutest). Esse
pacote facilita em muitos passos a instalação, apesar de limitar as escolhas
possíveis. Para funcionar com o Julia, é preciso que o CUTEst tenha bibliotecas
dinâmicas, e o homebrew-cutest já faz isso.

A partir daí, para instalar o CUTEst.jl, basta usar os comandos

In [10]:
Pkg.clone("https://github.com/abelsiqueira/CUTEst.jl")
Pkg.checkout("CUTEst", "fix/issue4")

INFO: Cloning CUTEst from https://github.com/abelsiqueira/CUTEst.jl


0.009645 seconds (64.57 k allocations: 6.708 MB)
  0.010131 seconds (42.53 k allocations: 4.557 MB, 34.79% gc time)


LoadError: LoadError: CUTEst already exists
while loading In[10], in expression starting on line 1

Os detalhes dessa instalação no Linux podem ser vistos
[neste post](http://abelsiqueira.github.io/blog/cutest/julia/sifdecode/optimization/2015/10/01/installing-cutest-and-cutest.jl.html).

In [11]:
using CUTEst

In [12]:
nlp = CUTEstModel("HS35")

Minimization problem HS35
nvar = 3, ncon = 1 (1 linear)


In [13]:
println(nlp)

Minimization problem 

1x3 Array{Float64,2}:
 0.0  0.0  0.0

HS35
nvar = 3, ncon = 1 (1 linear)
lvar = 
uvar = 

1x3 Array{Float64,2}:
 1.0e20  1.0e20  1.0e20

1x1 Array{Float64,2}:
 0.0

1x1 Array{Float64,2}:
 1.0e20

1x3 Array{Float64,2}:
 0.5  0.5  0.5

1x1 Array{Float64,2}:
 0.0

1x1 Array{Int64,2}:
 1


lcon = 
ucon = 
x0 = 
y0 = 
nnzh = 5
nnzj = 3
linear constraints:    



Existem várias funções do CUTEst, e por enquanto ainda não temos a documentação
completa de todas. No entanto, as funções originais do CUTEst estão presentes,
e em alguns casos, com mais de uma interface.

Por exemplo, sabemos que a função `cfn` existe. Podemos fazer

In [14]:
?cfn

# cfn

The cfn subroutine evaluates the value of the objective function and general constraint functions of the problem decoded from a SIF file by the script sifdecoder at the point X. The problem under consideration is to minimize or maximize an objective function f(x) over all x ∈ Rn subject to general equations ci(x)=0, (i ∈ 1,...,mE), general inequalities ci(x)≤ci(x)≤ci(x), (i ∈ mE+1,...,m), and simple bounds xl≤x≤xu. The objective function is group-partially separable and all constraint functions are partially separable.

This help was generated automatically and may contain errors. For more information, run the shell command

```
man cutest_cfn
```

Usage:

```
cfn(io_err, n, m, x, f, c)
```

  * io_err:  [OUT] Array{Cint, 1}
  * n:       [IN] Array{Cint, 1}
  * m:       [IN] Array{Cint, 1}
  * x:       [IN] Array{Cdouble, 1}
  * f:       [OUT] Array{Cdouble, 1}
  * c:       [OUT] Array{Cdouble, 1}

```
f, c = cfn(n, m, x)
```

  * n:       [IN] Int
  * m:       [IN] Int
  * x:       [IN] Array{Float64, 1}
  * f:       [OUT] Float64
  * c:       [OUT] Array{Float64, 1}

```
f = cfn!(n, m, x, c)
```

  * n:       [IN] Int
  * m:       [IN] Int
  * x:       [IN] Array{Float64, 1}
  * f:       [OUT] Float64
  * c:       [OUT] Array{Float64, 1}

```
f, c = cfn(nlp, x)
```

  * nlp:     [IN] CUTEstModel
  * x:       [IN] Array{Float64, 1}
  * f:       [OUT] Float64
  * c:       [OUT] Array{Float64, 1}

```
f = cfn!(nlp, x, c)
```

  * nlp:     [IN] CUTEstModel
  * x:       [IN] Array{Float64, 1}
  * f:       [OUT] Float64
  * c:       [OUT] Array{Float64, 1}


Agora, podemos escolher o que usar. Nesse caso, o mais simples é

In [15]:
x = nlp.meta.x0
fx, c = cfn(nlp, x)

search: cfn cfn! cfunction IntrinsicFunction broadcast_function cutest_finalize



(2.25,[1.0])

Note que *cfn* é uma função para problemas com restrição.
Para problema irrestritos devemos usar *ufn*.
Vamos testar com o problema *BARD*.
Para isso, devemos finalizar o problema atual.

In [16]:
cutest_finalize(nlp)

In [17]:
nlp = CUTEstModel("BARD")

Minimization problem BARD
nvar = 3, ncon = 0 (0 linear)


In [18]:
fx = ufn(nlp, nlp.meta.x0)

41.68169586167801

In [19]:
println(nlp)

Minimization problem 

1x3 Array{Float64,2}:
 -1.0e20  -1.0e20  -1.0e20

1x3 Array{Float64,2}:
 1.0e20  1.0e20  1.0e20

1x0 Array{Float64,2}

1x0 Array{Float64,2}

1x3 Array{Float64,2}:
 1.0  1.0  1.0

1x0 Array{Float64,2}

Essas funções são melhorias sobre as funções originais do CUTEst,
no entanto, também é possível acessar as funções originais.

In [20]:
?ufn

# ufn

The ufn subroutine evaluates the value of the objective function of the problem decoded from a SIF file by the script sifdecoder at the point X. The problem under consideration is to minimize or maximize an objective function f(x) over all x ∈ Rn subject to the simple bounds xl≤x≤xu. The objective function is group-partially separable.

This help was generated automatically and may contain errors. For more information, run the shell command

```
man cutest_ufn
```

Usage:

```
ufn(io_err, n, x, f)
```

  * io_err:  [OUT] Array{Cint, 1}
  * n:       [IN] Array{Cint, 1}
  * x:       [IN] Array{Cdouble, 1}
  * f:       [OUT] Array{Cdouble, 1}

```
f = ufn(n, x)
```

  * n:       [IN] Int
  * x:       [IN] Array{Float64, 1}
  * f:       [OUT] Float64

```
f = ufn(nlp, x)
```

  * nlp:     [IN] CUTEstModel
  * x:       [IN] Array{Float64, 1}
  * f:       [OUT] Float64


In [21]:
io_err = Cint[0]
n = Cint[nlp.meta.nvar]
x = nlp.meta.x0
fx = [0.0]
ufn(io_err, n, x, fx)

In [22]:
println(fx)

A simplificação deixa o código muito mais limpo, mas a versão original é mais
rápida.

Além disso, temos também algumas funções feitas para facilitar ainda
mais o uso do CUTEst.

In [23]:
fx = obj(nlp, nlp.meta.x0)

41.68169586167801

Essa função verifica se deve ser usado ufn ou cfn dentro do código.
Por isso será mais lenta. No entanto, é muito mais prática.

Outros exemplos

In [24]:
cutest_finalize(nlp); nlp = CUTEstModel("BT3")
cx = cons(nlp, nlp.meta.x0)

BARD
nvar = 3, ncon = 0 (0 linear)
lvar = 
uvar = 
lcon = 
ucon = 
x0 = 
y0 = 
nnzh = 6
nnzj = 0

search: ufn ufn! UTF8String takebuf_string cutest_finalize UTF32String

[41.68169586167801]


3-element Array{Float64,1}:
 80.0
  0.0
  0.0

In [25]:
Hx = hess(nlp, nlp.meta.x0)

5x5 sparse matrix with 9 Float64 entries:
	[1, 1]  =  2.0
	[2, 1]  =  -2.0
	[1, 2]  =  -2.0
	[2, 2]  =  4.0
	[3, 2]  =  2.0
	[2, 3]  =  2.0
	[3, 3]  =  2.0
	[4, 4]  =  2.0
	[5, 5]  =  2.0

In [26]:
Wx = hess(nlp, nlp.meta.x0, nlp.meta.y0)

5x5 sparse matrix with 9 Float64 entries:
	[1, 1]  =  2.0
	[2, 1]  =  -2.0
	[1, 2]  =  -2.0
	[2, 2]  =  4.0
	[3, 2]  =  2.0
	[2, 3]  =  2.0
	[3, 3]  =  2.0
	[4, 4]  =  2.0
	[5, 5]  =  2.0

In [27]:
Jx = jac(nlp, nlp.meta.x0)

3x5 sparse matrix with 7 Float64 entries:
	[1, 1]  =  1.0
	[1, 2]  =  3.0
	[3, 2]  =  1.0
	[2, 3]  =  1.0
	[2, 4]  =  1.0
	[2, 5]  =  -2.0
	[3, 5]  =  -1.0

In [28]:
cutest_finalize(nlp); nlp = CUTEstModel("ROSENBR")

Minimization problem ROSENBR
nvar = 2, ncon = 0 (0 linear)


## Usando com nosso algoritmo

Note que a função do CUTEst não segue o padrão da função que
usamos no nosso algoritmo. Mas isso pode ser facilmente resolvido.

In [29]:
f(x) = obj(nlp, x)
∇f(x) = grad(nlp, x)
x, fx, ef, k = metodo_gradiente3(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=100000)
println("x = $x, f(x) = $fx; ef = $ef; k = $k")

x = [0.9999892095536529,0.9999783777769542], f(x) = 1.1660551597130936e-10; ef = 0; k = 1449


Simples assim

In [30]:
function metodo_bfgs(f, ∇f, x₀; ϵ = 1e-4, α = 0.5, η = 0.5, kmax = 1000)
    B = eye(length(x₀))
    ef = 0
    x = copy(x₀)
    k = 0
    while norm(∇f(x)) > ϵ
        d = -B\∇f(x)
        dt∇f = dot(d,∇f(x))
        t = 1.0
        while f(x + t*d) > f(x) + α*t*dt∇f
            t *= η
        end
        s = t*d
        y = ∇f(x+s)- ∇f(x)
        x = x + s
        
        k += 1
        if k > kmax
            ef = 1
            break
        end
        
        if dot(s,y) > 0.0
            B = B + y*y'/dot(y,s) - B*s*s'*B/dot(s,B*s)
        end
    end
    return x, f(x), ef, k
end

metodo_bfgs (generic function with 1 method)

In [31]:
x, fx, ef, k = metodo_bfgs(f, ∇f, nlp.meta.x0, ϵ=1e-5, kmax=100000)
println("x = $x, f(x) = $fx; ef = $ef; k = $k")

x = [0.9999996948881412,0.9999993862916682], f(x) = 9.430756494150018e-14; ef = 0; k = 34


# Exercícios

## Melhore o BFGS

Faça algumas melhorias para evitar cálculos desnecessários.
**Não mude deste BFGS para o explícito.**

## Região de confiança

Mude o método para usar região de confiança.

## Método de Newton

Implemente o método de Newton

## Método restrito

Implemente algum método com restrições

## Método sem derivadas

Implemente algum método sem derivada